## moomoo_api.py Implementation

In [1]:
import os
import time
import subprocess
from dotenv import load_dotenv
import moomoo as moomoo
from moomoo.trade.open_trade_context import OpenSecTradeContext
import pandas as pd
import yfinance as yf
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import yfinance as yf
from typing import Optional, Dict, List


def start_opend_headless():
    opend_path = r"moomoo_OpenD_9.6.5618_Windows\OpenD.exe"
    try:
        if os.name == 'nt': # Windows
            process = subprocess.Popen([opend_path], creationflags=subprocess.CREATE_NEW_CONSOLE)
        else: # Linux/macOS
            process = subprocess.Popen([opend_path])
        print(f"OpenD started with PID: {process.pid}")
        if process:
             print("Connecting to OpenD via API...")
    except FileNotFoundError:
        print(f"Error: OpenD executable not found at {opend_path}")
        return None
    
    time.sleep(5)  # Wait for OpenD to initialize
    
    # Get the RSA key path from environment variables
    load_dotenv()
    key_path = os.getenv("KEY_PATH")
    # 1. Configure the RSA private key file globally
    moomoo.SysConfig.set_init_rsa_file(key_path)
    # 2. Create the trade context and enable encryption
    # is_encrypt=True encrypts using RSA key above
    trade_ctx = OpenSecTradeContext(
        host='127.0.0.1',
        port=11111,
        is_encrypt=True,
        security_firm="FUTUSG"
        )
    return trade_ctx, process

In [2]:
def account_list(trade_obj: OpenSecTradeContext):
    ret, data = trade_obj.get_acc_list()
    if ret == moomoo.RET_OK:
        return data
    else:
        raise Exception('get_acc_list error: ', data)
    
def account_info(trade_obj: OpenSecTradeContext):
    ret, data = trade_obj.accinfo_query(trd_env="REAL",refresh_cache=True,currency="SGD")
    if ret == moomoo.RET_OK:
        return data
    else:
        raise Exception('accinfo_query error: ', data)
    
def get_positions(trade_obj: OpenSecTradeContext):
    ret, data = trade_obj.position_list_query(trd_env="REAL",refresh_cache=True)
    if ret == moomoo.RET_OK:
        return data
    else:
        raise Exception('position_list_query error: ', data)

def historical_account_cashflow(trade_obj: OpenSecTradeContext):
    all_cash_flow_data = pd.DataFrame()
    end_date = datetime.strptime('2023-08-07', '%Y-%m-%d')
    current_date = datetime.combine(date.today(), datetime.min.time())
    request_count = 0
    start_time = time.time()
    
    while end_date <= current_date:
        # Rate Limit Check: 20 requests per 30 seconds
        if request_count >= 20:
            elapsed = time.time() - start_time
            if elapsed < 30:
                wait_time = 30 - elapsed + 1 # Add 1s buffer
                print(f"Quota used. Waiting {wait_time:.2f}s...")
                time.sleep(wait_time)
            # Reset window
            request_count = 0
            start_time = time.time()

        date_str = current_date.strftime('%Y-%m-%d')
        ret, data = trade_obj.get_acc_cash_flow(clearing_date=date_str, trd_env="REAL")

        if ret == moomoo.RET_OK:
            if not data.empty:
                all_cash_flow_data = pd.concat([all_cash_flow_data, data], ignore_index=True)
            request_count += 1
            current_date -= timedelta(days=1)

        elif ret == moomoo.RET_ERROR:
            print(f"Error on {date_str}: {data}")
            time.sleep(30)
            start_time = time.time()
            request_count = 0

    return all_cash_flow_data
    
def get_historical_orders(trade_obj: OpenSecTradeContext):
    ret, data = trade_obj.history_order_list_query(start="2023-08-07 00:00:00",end=datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    if ret == moomoo.RET_OK:
        return data
    else:
        raise Exception('history_order_list_query error: ', data)

In [3]:
trade_obj, process = start_opend_headless()
acc_list = account_list(trade_obj)
acc_info = account_info(trade_obj)
positions = get_positions(trade_obj)
cashflow = historical_account_cashflow(trade_obj)
historical_orders = get_historical_orders(trade_obj)
trade_obj.close()
process.terminate()

OpenD started with PID: 10832
Connecting to OpenD via API...
2026-01-07 11:15:29,800 | 9852 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=102089581
Quota used. Waiting 30.40s...
Quota used. Waiting 30.29s...
Quota used. Waiting 30.35s...
Quota used. Waiting 30.46s...
Quota used. Waiting 30.45s...
Quota used. Waiting 30.49s...
Quota used. Waiting 30.37s...
Quota used. Waiting 30.36s...
Quota used. Waiting 30.20s...
Quota used. Waiting 30.43s...
Quota used. Waiting 29.66s...
Quota used. Waiting 30.41s...
Quota used. Waiting 30.44s...
Quota used. Waiting 30.45s...
Quota used. Waiting 30.22s...
Quota used. Waiting 30.29s...
Quota used. Waiting 30.33s...
Quota used. Waiting 30.40s...
Quota used. Waiting 30.09s...
Quota used. Waiting 30.42s...
Quota used. Waiting 30.47s...
Quota used. Waiting 30.46s...
Quota used. Waiting 30.40s...
Quota used. Waiting 30.44s...
Quota used. Waiting 30.39s...
Quota used. Waiting 30.38s...
Q

## Cleanup the Dataframes Extracted

In [4]:
def cleanup_acc_info(acc_info:pd.DataFrame):
    filter_list = ['total_assets','securities_assets', 'fund_assets','bond_assets','cash','pending_asset','frozen_cash','avl_withdrawal_cash','risk_status',
               'us_cash', 'us_avl_withdrawal_cash','usd_net_cash_power','usd_assets',
               'sg_cash', 'sg_avl_withdrawal_cash','sgd_net_cash_power', 'sgd_assets']
    acc_info = acc_info.loc[:, filter_list]
    columns_to_round = [
    'total_assets', 'securities_assets', 'fund_assets', 'bond_assets',
    'cash', 'pending_asset', 'frozen_cash', 'avl_withdrawal_cash',
    'us_cash', 'us_avl_withdrawal_cash', 'usd_net_cash_power',
    'usd_assets', 'sg_cash', 'sg_avl_withdrawal_cash',
    'sgd_net_cash_power', 'sgd_assets'
    ]
    acc_info.loc[:, columns_to_round] = acc_info[columns_to_round].round(2)
    return acc_info

def get_cash(acc_info: pd.DataFrame) -> float:

    cash_columns = ['fund_assets', 'cash', 'pending_asset', 'frozen_cash', 'avl_withdrawal_cash',]
    return acc_info[cash_columns].sum(axis=1)[0].round(2)

def get_total_assets(acc_info: pd.DataFrame) -> float:
    return acc_info['total_assets'][0].round(2)

def get_securities(acc_info: pd.DataFrame) -> float:
    return acc_info['securities_assets'][0].round(2)

def get_bonds(acc_info: pd.DataFrame) -> float:
    return acc_info['bond_assets'][0].round(2)

def extract_ticker(code: str) -> str:
    pattern = r"^[A-Z]+\.(?P<ticker>[A-Z0-9]+).*"
    match = re.match(pattern, code)
    if match:
        return match.group("ticker")
    else:
        return None
    
def cleanup_positions(positions:pd.DataFrame):
    pos_filter_col = ['code', 'stock_name', 'position_market', 'qty','diluted_cost','market_val','nominal_price', 'pl_ratio','pl_val','today_pl_val','currency']
    positions = positions.loc[:, pos_filter_col]
    rounding_dict = {
                    'qty': 2,
                    'diluted_cost': 2,
                    'market_val': 2,
                    'nominal_price': 2,
                    'pl_ratio': 2,
                    'pl_val': 2,
                    'today_pl_val': 2
                    }
    positions = positions.round(rounding_dict)
    positions = positions.sort_values(by='market_val', ascending=False).reset_index(drop=True)
    positions.rename(columns={'code': 'Symbol',
                            'stock_name':'Name',
                            'position_market':'Market',
                            'qty':'Quantity',
                            'diluted_cost':'Diluted_Cost',
                            'market_val':'Market_Value',
                            'nominal_price':'Current_Price',
                            'pl_ratio':'P_L_Percent',
                            'pl_val':'P_L',
                            'today_pl_val':"Today_s_P_L",
                            'currency':'Currency'}, inplace=True)
    positions['Symbol'] = positions['Symbol'].apply(extract_ticker)
    return positions



def get_exchange_rate(from_currency:str,to_currency:str):
    if from_currency == to_currency:
        return 1.0
    elif to_currency == 'USD':
        ticker = f"{from_currency}=X"
    else:
        ticker = f"{from_currency}{to_currency}=X"
    price_data = yf.download(tickers=ticker, period='2d',
                             auto_adjust=True,
                             interval='1m',
                             progress=False,
                             prepost=True)
    if price_data.empty:
        print("Warning: Could not download share price data.")
        return 

    # Extract the most recent price for each ticker
    # Forward-fill to propagate the last valid price, then select the last row.
    # This handles cases where some tickers may not have traded in the last minute.
    latest_price = price_data['Close'].ffill().iloc[-1].round(decimals=3).item()
    return latest_price

def convert_currency(value, from_currency:str, to_currency:str) -> Optional[float]:
    """Converts a given amount from one currency to another."""
    rate = get_exchange_rate(from_currency, to_currency)
    if rate:
        converted_amount = value * rate
        return round(converted_amount, 2)
    else:
        return None

def update_portfolio_percentage(pos: pd.DataFrame, total_assets: float) -> None:
    if total_assets == 0:
        pos['Portfolio_Percent'] = 0.0
    else:
        pos['Portfolio_Percent'] = (convert_currency(pos['Market_Value'],"USD","SGD") / total_assets * 100).round(2)
        pos['Portfolio_Percent'] = pos['Portfolio_Percent'].apply(
            lambda x: f"{x:.2f}%"
        )

def cleanup_historical_orders(historical_orders:pd.DataFrame):
    historical_orders = historical_orders.loc[historical_orders['order_status'] == 'FILLED_ALL', 
                                          ['code', 'stock_name','order_market', 'trd_side','qty', 'price','currency','updated_time']]
    historical_orders.rename(columns={'code': 'Symbol',
                            'stock_name':'Name',
                            'order_market':'Market',
                            'trd_side':'Buy_Sell',
                            'qty':'Quantity',
                            'price':'Current_Price',
                            'currency':'Currency',
                            'updated_time':'date_time'}, inplace=True)
    historical_orders['Symbol'] = historical_orders['Symbol'].apply(extract_ticker)
    return historical_orders

def cleanup_cashflow(cashflow:pd.DataFrame):
    cashflow_filter = ['clearing_date','currency','cashflow_type','cashflow_direction','cashflow_amount','cashflow_remark']
    cashflow = cashflow.loc[:, cashflow_filter]
    cashflow.rename(columns={'clearing_date':'Date',
                            'currency':'Currency',
                            'cashflow_type':'Type',
                            'cashflow_direction':'in_out',
                            'cashflow_amount':'Amount',
                            'cashflow_remark':'Remark'}, inplace=True)
    cashflow['Amount'] = cashflow['Amount'].round(2)
    return cashflow

In [5]:
acc_info = cleanup_acc_info(acc_info)
positions = cleanup_positions(positions)
update_portfolio_percentage(positions, get_total_assets(acc_info))
historical_orders = cleanup_historical_orders(historical_orders)
cashflow = cleanup_cashflow(cashflow)

display(acc_info)
print ("Total Assets: ",get_total_assets(acc_info))
print ("Securities: ",get_securities(acc_info))
print ("Cash: ",get_cash(acc_info))
print ("Bonds: ",get_bonds(acc_info))
display(positions)
display(historical_orders)
display(cashflow)

,total_assets,securities_assets,fund_assets,bond_assets,cash,pending_asset,frozen_cash,avl_withdrawal_cash,risk_status,us_cash,us_avl_withdrawal_cash,usd_net_cash_power,usd_assets,sg_cash,sg_avl_withdrawal_cash,sgd_net_cash_power,sgd_assets
0,55669.14,55589.79,79.36,0.0,1582.85,0.0,0.0,0.0,LEVEL5,1235.91,0.0,1297.25,43405.35,0.0,0.0,0.01,0.01


Total Assets:  55669.14
Securities:  55589.79
Cash:  1662.21
Bonds:  0.0


,Symbol,Name,Market,Quantity,Diluted_Cost,Market_Value,Current_Price,P_L_Percent,P_L,Today_s_P_L,Currency,Portfolio_Percent
0,NVDA270115C230000,NVDA 270115 230.00C,US,7.0,23.50,15435.00,22.05,-6.17,-1015.00,0.00,USD,35.52%
1,GOOG,Alphabet-C,US,25.0,182.76,7858.75,314.35,72.01,3289.88,-5.00,USD,18.08%
2,AMZN260918C250000,AMZN 260918 250.00C,US,3.0,21.50,7830.00,26.10,21.40,1380.00,0.00,USD,18.02%
3,ASML,ASML Holding,US,3.0,400.48,3714.03,1238.01,209.13,2512.58,-12.54,USD,8.55%
4,SOFI,SoFi Technologies,US,124.0,25.13,3341.80,26.95,7.23,225.15,-3.72,USD,7.69%
5,ASTS,AST SpaceMobile,US,30.0,42.71,2877.60,95.92,124.58,1596.28,-47.10,USD,6.62%
6,V,Visa,US,8.0,326.80,2864.56,358.07,9.57,250.15,4.08,USD,6.59%
7,AMZN,Amazon,US,11.0,222.53,2646.38,240.58,8.11,198.53,-3.85,USD,6.09%
8,GRAB270115C5000,GRAB 270115 5.00C,US,20.0,1.88,2500.00,1.25,-33.58,-1265.00,0.00,USD,5.75%
9,HIMS,Hims & Hers Health,US,60.0,41.77,2128.80,35.48,-15.06,-377.60,1.20,USD,4.90%


,Symbol,Name,Market,Buy_Sell,Quantity,Current_Price,Currency,date_time
0,ASML,ASML Holding,US,SELL,1.0000,1237.00,USD,2026-01-06 10:46:41.833
1,META,Meta Platforms,US,BUY,0.0016,664.94,USD,2025-12-24 03:34:59.059
2,SOFI,SoFi Technologies,US,BUY,4.0000,26.50,USD,2025-12-12 11:16:16.738
3,META,Meta Platforms,US,BUY,1.0000,670.00,USD,2025-12-05 10:45:07.428
4,BMNR,Bitmine Immersion Technologies,US,BUY,5.0000,29.30,USD,2025-12-01 11:22:26.916
...,...,...,...,...,...,...,...,...
518,INTC,Intel,US,BUY,3.0000,31.03,USD,2024-07-03 09:30:34.574
519,NVDA,NVIDIA,US,BUY,2.0000,121.78,USD,2024-07-03 04:01:18.676
520,SE,Sea,US,SELL,1.0000,70.70,USD,2024-07-03 04:02:05.500
521,TSLA,Tesla,US,BUY,1.0000,209.89,USD,2024-07-02 04:00:57.474


,Date,Currency,Type,in_out,Amount,Remark
0,2025-12-30,USD,Fund Subscription,OUT,-0.39,Fund Subscription#CSOP USD Money Market Fund
1,2025-12-29,USD,Cash Dividend,IN,0.55,BMNR 55.00000000 SHARES DIVIDENDS 0.01000000 U...
2,2025-12-29,USD,Dividend Tax,OUT,-0.16,BMNR 55.00000000 SHARES WITHHOLDING TAX -0.003...
3,2025-12-24,USD,Fund Subscription,OUT,-0.03,Fund Subscription#CSOP USD Money Market Fund
4,2025-12-23,USD,Dividend Tax,OUT,-0.47,META 3.00000000 SHARES WITHHOLDING TAX -0.1574...
...,...,...,...,...,...,...
521,2023-08-07,SGD,Others,IN,20.00,Coupon Deposit
522,2023-08-07,SGD,Others,IN,20.00,Coupon Deposit
523,2023-08-07,SGD,Others,IN,20.00,Coupon Deposit
524,2023-08-07,SGD,Others,IN,5.00,Coupon Deposit


## Preprocess dataframes to be placed into SQL DB

In [15]:
def shares_df(positions:pd.DataFrame):
    return positions[positions['Symbol'].str.len() <= 4]

def options_df(positions:pd.DataFrame):
    return positions[positions['Symbol'].str.len() > 4]

def sum_of_mv(df:pd.DataFrame):
    converted_df = df.loc[:, ['Market_Value', 'Currency']].copy()
    converted_df['Market_Value'] = converted_df.apply(
                                lambda row: convert_currency(row['Market_Value'], row['Currency'], 'SGD'), axis=1
                                )
    return converted_df['Market_Value'].sum().round(2)


def initial_portfolio_snapshot_df(date_time: str, total_assets:float, shares_mv:float, options_mv:float, cash:float):
    data = {
        'date_time': [date_time],
        'total_assets': [total_assets],
        'stocks': [shares_mv],
        'options': [options_mv],
        'cash': [cash],
        'nav': [total_assets/1000],
        'units': [1000.0]
    }
    snapshot_df = pd.DataFrame(data)
    return snapshot_df
def initial_positions_df(positions:pd.DataFrame, date_time: str):
    positions_df = positions.copy()
    positions_df['date_time'] = date_time
    return positions_df

In [16]:
shares = shares_df(positions)
options = options_df(positions)
display(shares)
display(options)
shares_mv = sum_of_mv(shares)
options_mv = sum_of_mv(options)
print("Shares Market Value (SGD): ", shares_mv)
print("Options Market Value (SGD): ", options_mv)
cash = get_cash(acc_info)
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
initial_snapshot_df = initial_portfolio_snapshot_df(
    current_time,
    get_total_assets(acc_info),
    shares_mv,
    options_mv,
    cash
)
initial_positions_df = initial_positions_df(positions, current_time)

print("Cash (SGD): ", cash)
display(initial_snapshot_df)
display(initial_positions_df)

,Symbol,Name,Market,Quantity,Diluted_Cost,Market_Value,Current_Price,P_L_Percent,P_L,Today_s_P_L,Currency,Portfolio_Percent
1,GOOG,Alphabet-C,US,25.0,182.76,7858.75,314.35,72.01,3289.88,-5.00,USD,18.08%
3,ASML,ASML Holding,US,3.0,400.48,3714.03,1238.01,209.13,2512.58,-12.54,USD,8.55%
4,SOFI,SoFi Technologies,US,124.0,25.13,3341.80,26.95,7.23,225.15,-3.72,USD,7.69%
5,ASTS,AST SpaceMobile,US,30.0,42.71,2877.60,95.92,124.58,1596.28,-47.10,USD,6.62%
6,V,Visa,US,8.0,326.80,2864.56,358.07,9.57,250.15,4.08,USD,6.59%
7,AMZN,Amazon,US,11.0,222.53,2646.38,240.58,8.11,198.53,-3.85,USD,6.09%
9,HIMS,Hims & Hers Health,US,60.0,41.77,2128.80,35.48,-15.06,-377.60,1.20,USD,4.90%
10,META,Meta Platforms,US,3.0,648.28,1978.26,659.07,1.66,32.37,-4.65,USD,4.55%
11,BMNR,Bitmine Immersion Technologies,US,55.0,40.19,1816.65,33.03,-17.82,-393.86,38.50,USD,4.18%
12,DUOL,Duolingo,US,10.0,182.40,1814.30,181.43,-0.53,-9.70,24.90,USD,4.17%


,Symbol,Name,Market,Quantity,Diluted_Cost,Market_Value,Current_Price,P_L_Percent,P_L,Today_s_P_L,Currency,Portfolio_Percent
0,NVDA270115C230000,NVDA 270115 230.00C,US,7.0,23.50,15435.00,22.05,-6.17,-1015.00,0.0,USD,35.52%
2,AMZN260918C250000,AMZN 260918 250.00C,US,3.0,21.50,7830.00,26.10,21.40,1380.00,0.0,USD,18.02%
8,GRAB270115C5000,GRAB 270115 5.00C,US,20.0,1.88,2500.00,1.25,-33.58,-1265.00,0.0,USD,5.75%
16,GRAB270115C7500,GRAB 270115 7.50C,US,-20.0,0.96,-1120.00,0.56,41.51,795.00,0.0,USD,-2.58%
17,AMZN260918C280000,AMZN 260918 280.00C,US,-3.0,12.60,-4552.50,15.18,-20.44,-772.50,0.0,USD,-10.48%
18,NVDA270115C250000,NVDA 270115 250.00C,US,-7.0,18.20,-12036.99,17.20,5.52,703.01,0.0,USD,-27.70%


Shares Market Value (SGD):  43699.63
Options Market Value (SGD):  10319.11
Cash (SGD):  1662.21


,date_time,total_assets,stocks,options,cash,nav,units
0,2026-01-07 11:52:06,55669.14,43699.63,10319.11,1662.21,55.66914,1000.0


,Symbol,Name,Market,Quantity,Diluted_Cost,Market_Value,Current_Price,P_L_Percent,P_L,Today_s_P_L,Currency,Portfolio_Percent,date_time
0,NVDA270115C230000,NVDA 270115 230.00C,US,7.0,23.50,15435.00,22.05,-6.17,-1015.00,0.00,USD,35.52%,2026-01-07 11:52:06
1,GOOG,Alphabet-C,US,25.0,182.76,7858.75,314.35,72.01,3289.88,-5.00,USD,18.08%,2026-01-07 11:52:06
2,AMZN260918C250000,AMZN 260918 250.00C,US,3.0,21.50,7830.00,26.10,21.40,1380.00,0.00,USD,18.02%,2026-01-07 11:52:06
3,ASML,ASML Holding,US,3.0,400.48,3714.03,1238.01,209.13,2512.58,-12.54,USD,8.55%,2026-01-07 11:52:06
4,SOFI,SoFi Technologies,US,124.0,25.13,3341.80,26.95,7.23,225.15,-3.72,USD,7.69%,2026-01-07 11:52:06
5,ASTS,AST SpaceMobile,US,30.0,42.71,2877.60,95.92,124.58,1596.28,-47.10,USD,6.62%,2026-01-07 11:52:06
6,V,Visa,US,8.0,326.80,2864.56,358.07,9.57,250.15,4.08,USD,6.59%,2026-01-07 11:52:06
7,AMZN,Amazon,US,11.0,222.53,2646.38,240.58,8.11,198.53,-3.85,USD,6.09%,2026-01-07 11:52:06
8,GRAB270115C5000,GRAB 270115 5.00C,US,20.0,1.88,2500.00,1.25,-33.58,-1265.00,0.00,USD,5.75%,2026-01-07 11:52:06
9,HIMS,Hims & Hers Health,US,60.0,41.77,2128.80,35.48,-15.06,-377.60,1.20,USD,4.90%,2026-01-07 11:52:06


## Initialise DB

In [17]:
import sqlite3
def init_db():
    conn = sqlite3.connect('moomoo_portfolio.db')
    cursor = conn.cursor()

    # Create portfolio_snapshots table
    portfolio_snapshots_table ="""
    CREATE TABLE IF NOT EXISTS portfolio_snapshots (
        date_time TEXT PRIMARY KEY,
        total_assets REAL,
        stocks REAL,
        options REAL,
        cash REAL,
        nav REAL,
        units REAL
    )
    """
    # Create the Positions table
    positions_table = """
    CREATE TABLE IF NOT EXISTS positions (
        Symbol TEXT,
        Name TEXT,
        Market TEXT,
        Quantity REAL,
        Diluted_Cost REAL,
        Market_Value REAL,
        Current_Price REAL,
        P_L_Percent REAL,
        P_L REAL,
        Today_s_P_L REAL,
        Currency TEXT,
        Portfolio_Percent REAL,
        date_time TEXT,
        FOREIGN KEY (date_time) REFERENCES portfolio_snapshots (date_time)
    )
    """
    # Create historical_orders table
    historical_orders_table = """
    CREATE TABLE IF NOT EXISTS historical_orders (
        Symbol TEXT,
        Name TEXT,
        Market TEXT,
        Buy_Sell TEXT,
        Quantity REAL,
        Current_Price REAL,
        Currency TEXT,
        date_time TEXT
    )
    """
    # Create cashflow table
    cashflow_table = """
    CREATE TABLE IF NOT EXISTS cashflow (
        Date TEXT,
        Currency TEXT,
        Type TEXT,
        in_out TEXT,
        Amount REAL,
        Remark TEXT
    )
    """
    cursor.execute(portfolio_snapshots_table)
    cursor.execute(positions_table)
    cursor.execute(historical_orders_table)
    cursor.execute(cashflow_table)




    conn.commit()
    conn.close()

def insert_dataframe(df:pd.DataFrame, table_name:str):
    conn = sqlite3.connect('moomoo_portfolio.db')
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()

In [18]:
init_db()
insert_dataframe(initial_snapshot_df, 'portfolio_snapshots')
insert_dataframe(initial_positions_df, 'positions')
insert_dataframe(historical_orders, 'historical_orders')
insert_dataframe(cashflow, 'cashflow')

In [ ]:
'''
shares = positions[positions['Symbol'].str.len() <= 4]
options = positions[positions['Symbol'].str.len() > 4]
display(shares)
display(options)
'''

In [ ]:
'''
def update_current_share_price(shares: pd.DataFrame) -> pd.DataFrame:
    tickers = shares['Symbol'].dropna().unique().tolist()
    if not tickers:
        return shares
    price_data = yf.download(tickers=tickers, period='2d',
                             interval='1m',
                             progress=False,
                             prepost=True)

    if price_data.empty:
        print("Warning: Could not download share price data.")
        return shares

    # Extract the most recent price for each ticker
    # Forward-fill to propagate the last valid price, then select the last row.
    # This handles cases where some tickers may not have traded in the last minute.
    latest_prices = price_data['Close'].ffill().iloc[-1].round(decimals=3)
    price_map = latest_prices.to_dict() if isinstance(latest_prices, pd.Series) else {tickers[0]: latest_prices}
    shares['Current Price'] = shares['Symbol'].map(price_map)
    return shares
def get_price_from_custom_format(custom_option_string: str) -> Optional[float]:
    """
    Parses a custom option string, finds the contract in the yfinance
    option chain, and returns its last traded price.

    Args:
        custom_option_string: A string in the format "TICKER YYMMDD STRIKE_PRICE(C/P)"
                              e.g., "AMZN 260918 195.00C"

    Returns:
        The last price as a float, or None if not found or an error occurs.
    """
    try:
        # 1. Parse the string using a robust regular expression
        pattern = r'^(?P<ticker>[A-Z]+)\s+(?P<date>\d{6})\s+(?P<strike>[\d.]+)(?P<type>[CP])$'
        match = re.match(pattern, custom_option_string.strip())

        if not match:
            print(f"Warning: Could not parse symbol '{custom_option_string}'")
            return None

        parts = match.groupdict()
        underlying_ticker = parts['ticker']
        exp_date_str = datetime.strptime(parts['date'], '%y%m%d').strftime('%Y-%m-%d')
        strike_price = float(parts['strike'])
        option_type = parts['type']

        # 2. Fetch the option chain for the specific expiration date
        ticker_obj = yf.Ticker(underlying_ticker)
        if option_type == 'C':
            chain = ticker_obj.option_chain(exp_date_str).calls
        else:
            chain = ticker_obj.option_chain(exp_date_str).puts

        # 3. Find the specific contract by its strike price
        contract = chain[chain['strike'] == strike_price]

        # 4. Extract the price if the contract was found
        if not contract.empty:
            return contract['lastPrice'].iloc[0].round(decimals=3)
        else:
            print(f"Warning: Contract for '{custom_option_string}' not found in the option chain.")
            return None

    except Exception as e:
        print(f"Warning: An error occurred for '{custom_option_string}'. Reason: {e}")
        return None

def get_option_prices_from_list(option_list: List[str]) -> Dict[str, Optional[float]]:
    """
    Takes a list of custom option strings and returns a dictionary
    mapping each option to its current price.
    """
    price_results = {}
    for option_string in option_list:
        price_results[option_string] = get_price_from_custom_format(option_string)
    return price_results

def update_option_prices(options: pd.DataFrame) -> pd.DataFrame:
    """
    Fetches current prices for a DataFrame of options.
    """
    contracts = options['Name'].to_list()
    price_dictionary = get_option_prices_from_list(contracts)
    options['Current Price'] = options['Name'].map(price_dictionary)
    return options

'''

In [ ]:
'''
shares = update_current_share_price(shares)
options = update_option_prices(options)
display(shares)
display(options)
'''

In [ ]:
'''
portfolio_df = pd.concat([shares, options], ignore_index=True)
display(portfolio_df)
'''

In [ ]:
'''
def update_market_val(portfolio_df: pd.DataFrame) -> None:
    # 1. Create a boolean mask: a Series of True/False for each row.
    is_option = portfolio_df['Symbol'].str.len() > 4

    # 2. Use np.where to create a new Series with the correct quantity for each row.
    effective_quantity = np.where(
        is_option,                      # Condition: For each row, is it an option?
        portfolio_df['Quantity'] * 100, # Value if True: Use Quantity * 100
        portfolio_df['Quantity']        # Value if False: Use just the Quantity
    )

    # 3. Use the new 'effective_quantity' for the final calculation.
    portfolio_df['Market Value'] = (portfolio_df['Current Price'].fillna(0) * effective_quantity).round(2)

def update_pl(portfolio_df: pd.DataFrame) -> None:
    is_option = portfolio_df['Symbol'].str.len() > 4
    effective_quantity = np.where(is_option, 
                                  portfolio_df['Quantity'] * 100, 
                                  portfolio_df['Quantity'])
    portfolio_df['P/L'] = ((portfolio_df['Current Price'].fillna(0) - portfolio_df['Diluted Cost'].fillna(0)) * effective_quantity).round(2)

def update_pl_ratio(portfolio_df: pd.DataFrame) -> None:
     # Use np.divide to safely handle potential division by zero
    pl_ratio_numeric = np.divide(portfolio_df['Current Price'] - portfolio_df['Diluted Cost'], portfolio_df['Diluted Cost']) * 100
    portfolio_df['P/L %'] = pl_ratio_numeric.round(2)
    portfolio_df['P/L %'] = portfolio_df['P/L %'].apply(
        lambda x: f"+{x:.2f}%" if x > 0 else f"{x:.2f}%"
    )
'''

'''
update_market_val(positions)
update_pl(positions)
update_pl_ratio(positions)
'''


In [ ]:

''' 
def plot_asset_class_allocation(portfolio_df: pd.DataFrame) -> None:
    """
    Plots a bar chart showing the total market value in Shares vs. Options.
    """
    # Create a temporary 'Asset Type' column for grouping
    asset_classes = {
            'Shares': shares_in_sgd,
            'Options': options_in_sgd,
            'Cash': get_cash(acc_info),
            'Bonds': get_bond_assets(acc_info)
            }
    filtered_assets = {k: v for k, v in asset_classes.items() if v > 0}

    plt.figure(figsize=(8, 8))
    colours = ['#4c72b0', '#c44e52', '#55a868', '#8172b3', '#ccb974', '#64b5cd']
    plt.pie(
            filtered_assets.values(),
            labels=filtered_assets.keys(),
            autopct='%1.1f%%',
            startangle=90,
            explode=[0.05] * (len(filtered_assets.keys())), # Explode all slices slightly for better visibility
            colors=colours
            )
    plt.title('Asset Class Allocation by Market Value', fontsize=16)
    plt.ylabel('') # Hide the y-label as it's not needed for a pie chart
    plt.show()

plot_asset_class_allocation(portfolio_df)
'''

In [ ]:
'''
def plot_portfolio_composition(portfolio_df: pd.DataFrame) -> None:
    

    # Sort by Market Value to make the chart easier to read
    sorted_df = portfolio_df.sort_values(by='Market Value', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Market Value', y='Name', data=sorted_df, hue='Name', palette='Dark2', legend=False)
    plt.title('Portfolio Composition by Market Value', fontsize=16)
    plt.xlabel('Market Value ($)')
    plt.ylabel('Name')
    plt.gca().get_xaxis().set_major_formatter(
        plt.FuncFormatter(lambda x, p: f'${x:,.0f}')
    )
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()
plot_portfolio_composition(positions)

def plot_pl_by_position(portfolio_df: pd.DataFrame) -> None:
    """
    Plots a bar chart showing the P/L for each position, colored by profit or loss.
    """
    # Sort by P/L for a more organized chart
    sorted_df = portfolio_df.sort_values(by='P/L', ascending=False)
    
    # Create a color palette for the bars
    colors = ["#00d63d" if x > 0 else "#dd2a13" for x in sorted_df['P/L']]
    
    plt.figure(figsize=(12, 8))
    sns.barplot(
        x='P/L',
        y='Name',
        data=sorted_df,
        hue='Name',
        palette=colors,
        legend=False
    )
    plt.title('Profit/Loss by Position', fontsize=16)
    plt.xlabel('Profit/Loss ($)')
    plt.ylabel('Stock Name')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()
    
plot_pl_by_position(positions)
'''
